<font size=8 face="Courier"><b>Eigendecomposition Source Code

In [ ]:
print("Loading: Eigendecomposition_Source_Code.ipynb...")

To do:

- Add comments to rotation code<font color="green"> --Done!
- Create randomized first 5 randomized eigenvectors plot in both neuron layout view and eigenvector plots (basis)<font color="green"> --Done!
- add seed option for randomizer<font color="green"> --Done!

# <font color="gray">Import Stuff

In [ ]:
import pytz
import numpy as np                                                    # Packages for data analysis
import pandas as pd
import random as rand
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as patches
from matplotlib.patches import Patch, Circle
from matplotlib.lines import Line2D
from scipy.ndimage import gaussian_filter1d
from datetime import datetime
import braingeneers                                                   # Braingeneers code
from braingeneers.analysis.analysis import SpikeData, read_phy_files, load_spike_data, burst_detection, randomize_raster
import braingeneers.data.datasets_electrophysiology as ephys
import ipywidgets as ipw
from ipywidgets import interact, interactive, fixed, interact_manual  # package for interactive widgets from IPython.display import HTML, display, Javascript, clear_output,
from IPython.display import HTML, display, Javascript, clear_output

In [ ]:
data_folders = !ls /home/jovyan/work/Human_Hippocampus/data/ephys

In [ ]:
Timezone = pytz.timezone("America/Los_Angeles")

In [ ]:
# OTHER PATHS:
# 2023-04-03-hc328b_ChR2
# 2023-04-04-e-hc328_hckcr1-2_040423_recs
# 2023-04-05-e-hc328_040523
# 2023-05-09-e-hc52_18763
# 2023-05-10-e-hc52_18790

# <font color="blue">Settings

Loads recording (ephys) data from a set of paths

In [ ]:
def loadData(FolderName):
    global folder;
    folder = FolderName
    filesTemp = !ls /home/jovyan/work/Human_Hippocampus/data/ephys/{folder}/derived/kilosort2/*curated*
    
    files = []
    for file in filesTemp: 
        namescomp = file.split(f"/home/jovyan/work/Human_Hippocampus/data/ephys/{folder}/derived/kilosort2/")
        files.append(str(namescomp[1]))
    
    interact_manual( loadFile, FileName=files )
    

In [ ]:
def loadFile(FileName):
    global file; global sd;
    
    file = FileName
    print(f"Loading {file} from dataset: {folder}...")
    #experiment = "help"
    #sd = load_spike_data(folder, basepath='../data/')
    #sd = load_spike_data(f"../data/ephys/{folder}/derived/kilosort2/{file}")
    #sd = load_spike_data(folder, experiment=file, basepath=f'../data/ephys/')
    sd = read_phy_files(f"../data/ephys/{folder}/derived/kilosort2/{file}") 
    
    now = datetime.now(Timezone)
    printNow = now.strftime("%Y/%m/%d %H:%M:%S")

    print(f"Done at: {printNow}")

In [ ]:
#interact_manual( loadData, FolderName=data_folders)

Allows user to select display color 

In [ ]:
#Default Color Selection:
plot_color = "magma"

In [ ]:
colors = ['viridis', 'plasma', 'inferno', 'magma', 'cividis', 'binary', 'bone', 'pink', 'spring', 'summer', 'autumn', 'winter', 'cool', 'Wistia', 'hot', 'afmhot', 'RdBu']

other color schecmes can be found at https://matplotlib.org/stable/tutorials/colors/colormaps.html

In [ ]:
def ColorSelect(selection):
    global plot_color;
    plot_color = selection
    print(f'color selection: {plot_color}')

In [ ]:
#interact_manual( ColorSelect, selection=colors )

# Functions

In [ ]:
### Scratch Functions (cleanup later)

def correlation(sd):
    # Correlation
    corr = np.zeros((sd.N,sd.N)) #inds by inds
    
    dense_raster = sd.raster(bin_size=1) # in ms
    sigma = 5                            # Blur it
    dense_raster = gaussian_filter1d(dense_raster.astype(float),sigma=sigma)
    corr=np.corrcoef( dense_raster )
        
    return corr;
    
def plot_raster(sd, title="Spike Raster", l1=-10, l2=False, xsize=10, ysize=6, analize=False):
    if l2==False:
        l2 = sd.length / 1000 + 10
    
    idces, times = sd.idces_times()
    
    if analize == True:
        # Get population rate for everything
        pop_rate = sd.binned(bin_size=1)  # in ms
        # Lets smooth this to make it neater
        sigma = 5
        pop_rate_smooth = gaussian_filter1d(pop_rate.astype(float), sigma=sigma)
        t = np.linspace(0, sd.length, pop_rate.shape[0]) / 1000

        # Determine the stop_time if it's not provided
        if l2 is None:
            l2 = t[-1]

        # Filter times and idces within the specified start and stop times
        mask = (times >= l1 * 1000) & (times <= l2 * 1000)
        times = times[mask]
        idces = idces[mask]

    fig, ax = plt.subplots(figsize=(xsize, ysize))
    fig.suptitle(title)
    ax.scatter(times/1000,idces,marker='|',s=1)
    
    if analize == True:
        ax2 = ax.twinx()
        ax2.plot(t, pop_rate_smooth, c='r')
        ax2.set_ylabel('Firing Rate')
        
    ax.set_xlabel("Time(s)")
    ax.set_ylabel('Unit #')
    plt.xlim(l1, l2)
    plt.show()
    
def eigenvalues_eigenvectors(A):
    W, U = np.linalg.eigh(A)
    # The rank of A can be no greater than the smaller of its
    # dimensions, so cut off the returned values there.
    rank = min(*A.shape)
    U = U[:,-rank:]
    sgn = (-1)**(U[0,:] < 0)
    # Also reverse the order of the eigenvalues because eigh()
    # returns them in ascending order but descending makes more sense.
    return W[-rank:][::-1], (U*sgn[np.newaxis,:])[:, ::-1]

def toeplitz(M):
    K = M.shape[0]
    topz = np.zeros_like(M)
    for i in range(1 - K, K):
        meantopz = np.mean(np.diagonal(M, offset=i))
        topz += np.eye(K, K, k=i) * meantopz
        topz += np.eye(K, K, k=-i) * meantopz
    return topz
        
def get_sttc(sd):
    sttc = sd.spike_time_tilings()
    return sttc
    
def reconstruct(W, U, rank):
    Wd = np.diag(W[:rank])
    Ur = U[:, :rank]
    return Ur @ Wd @ Ur.T
    
def reconstruction_errors(A, ranks):
    norm = np.linalg.norm(A)
    W, U = eigenvalues_eigenvectors(A)
    return np.array([
        np.linalg.norm( reconstruct(W, U, rank) - A) / norm
        for rank in ranks])

def calculate_mean_firing_rates(spike_data):
    mean_firing_rates = []
    for neuron_spikes in spike_data.train:
        num_spikes = len(neuron_spikes)
        time_duration = spike_data.length / 1000  # Assuming spike times are in milliseconds
        firing_rate = num_spikes / time_duration
        mean_firing_rates.append(firing_rate)
    return np.array(mean_firing_rates)

# Subplots

Main command that plots all subplots:

## Correlation Methods Plot

In [ ]:
def corrMethPlots(Corr, STTC):
    fig, plot = plt.subplot_mosaic("AB", figsize=(14,7)) 
    
    # Plot Correlation Matrix
    pltA = plot["A"].imshow(Corr, cmap=plot_color) # plot STTC matrix
    plot["A"].set_title("Correlation")
    plot["A"].set_xlabel("Neuron index")
    plot["A"].set_ylabel("Neuron index" )
    fig.colorbar(pltA, ax=plot["A"], shrink=0.7)

    # Plot STTC matrix
    pltB = plot["B"].imshow(STTC, cmap=plot_color) # plot STTC matrix
    plot["B"].set_title("STTC")
    plot["B"].set_xlabel("Neuron index")
    plot["B"].set_ylabel("Neuron index" )
    fig.colorbar(pltB, ax=plot["B"], shrink=0.7)

## Eigenvector Analysis

In [ ]:
def EigenvectorAnalysis(corr, sttc):
    """figLayout = 
                AAABBB
                CCCDDD
                CCCDDD
                CCCDDD
                """
    
    figLayout = """
                AABB
                CCDD
                CCDD
                """
    
    fig, plot = plt.subplot_mosaic(figLayout, figsize=(16,8))
    fig.tight_layout(pad=5.0)
    
    # Plot Correlation basis
    plot["A"].stem(corr[:,0])
    plot["A"].set_xlabel('Observation Dimension')
    plot["A"].set_title('First Eigenvector of Correlation')    
    
    # Plot STTC basis
    plot["B"].stem(sttc[:,0])
    plot["B"].set_xlabel('Observation Dimension')
    plot["B"].set_title('First Eigenvector of STTC')    
    
    # Plot Correlation Matrix
    pltC = plot["C"].imshow(corr[:,:2*len(corr)].T, interpolation='none', cmap=plot_color)
    #plot["A"].gca().set_aspect('auto')
    plot["C"].set_ylabel('Eigenvector Number')
    plot["C"].set_xlabel('Observation Dimension')
    plot["C"].set_title('Eigenvectors of Correlation')
    fig.colorbar(pltC, ax=plot["C"], shrink=0.7)
    
    # Plot STTC matrix
    pltD = plot["D"].imshow(sttc[:,:2*len(sttc)].T, interpolation='none', cmap=plot_color)
    #plot["B"].gca().set_aspect('auto')
    plot["D"].set_ylabel('Eigenvector Number')
    plot["D"].set_xlabel('Observation Dimension')
    plot["D"].set_title('Eigenvectors of STTC')
    fig.colorbar(pltD, ax=plot["D"], shrink=0.7)

### <font color ="green"> Specialized Functions

In [ ]:
def plot_evectmatrix(corr, sttc):
    fig, plot = plt.subplot_mosaic("AB", figsize=(14,7))
    
    # Plot Correlation Matrix
    pltA = plot["A"].imshow(corr[:,:2*len(corr)].T, interpolation='none', cmap=plot_color)
    #plot["A"].gca().set_aspect('auto')
    plot["A"].set_ylabel('Eigenvector Number')
    plot["A"].set_xlabel('Observation Dimension')
    plot["A"].set_title('Eigenvectors of Correlation')
    fig.colorbar(pltA, ax=plot["A"], shrink=0.7)
    
    # Plot STTC matrix
    pltA = plot["B"].imshow(sttc[:,:2*len(sttc)].T, interpolation='none', cmap=plot_color)
    #plot["B"].gca().set_aspect('auto')
    plot["B"].set_ylabel('Eigenvector Number')
    plot["B"].set_xlabel('Observation Dimension')
    plot["B"].set_title('Eigenvectors of STTC')
    fig.colorbar(pltA, ax=plot["B"], shrink=0.7)

In [ ]:
def plot_basis(method):
    
    if method == "Correlation":
        Corr = correlation(sd)
        Wcorr, Ucorr = eigenvalues_eigenvectors(Corr)
        A = Ucorr
    else:
        STTC = get_sttc(sd)
        Wsttc, Usttc = eigenvalues_eigenvectors(STTC)
        A = Usttc
        
    plt.figure(figsize=(6,8))
    for i in range(5):
        if i: plt.xticks([])
        plt.subplot(5, 1, i+1)
        plt.stem(A[:,i])
    plt.xlabel('Observation Dimension')
    plt.suptitle('First 5 Eigenvectors', y=0.92)    

## Principle Component Analysis

In [ ]:
def PCAplots(corr, sttc):
    plt.figure(figsize=(7,5))
    plt.scatter(corr[0], corr[1]) # first two eigenvectors of correlation 
    plt.scatter(sttc[0], sttc[1]) # first two eigenvectors of STTC
    plt.legend(["Correlation Eigenvectors", "STTC Eigenvectors"])
    plt.title("Principle Components Analysis of Correlation Methods")
    plt.xlabel("PC1")
    plt.ylabel("PC2")

## Reconstructions

In [ ]:
def ReconstructPlots(Wcorr, Wsttc, Corr, STTC):
    fig, plot = plt.subplot_mosaic("AB", figsize=(15,5)) 
    
    # Eigenvalue spectrum ---------------------------------------------------------
    index = 1 + np.arange(len(Wcorr))
    plot["A"].semilogy(index, Wcorr, label='Correlation')
    plot["A"].plot(index, Wsttc, label='STTC')
    plot["A"].set_title('Eigenvalue Spectrum')
    plot["A"].set_xlabel('Index')
    plot["A"].set_ylabel('Eigenvalue')
    plot["A"].axvline(len(Wcorr)+1, ls=':', c='k')
    plot["A"].legend(loc="upper right");
    
    # Reconstruction Error --------------------------------------------------------
    errs_corr = reconstruction_errors(Corr, range(len(Corr)))      # Correlation
    errs_sttc = reconstruction_errors(STTC , range(len(STTC)))     # STTC   
    
    plot["B"].plot(errs_corr, label='Correlation Matrix')          # Correlation
    plot["B"].plot(errs_sttc, label='STTC Matrix')                 # STTC
    
    plot["B"].set_title("Reconstruction Error of Correlation Methods")
    plot["B"].set_xlabel('Components Used in Reconstruction')
    plot["B"].set_ylabel('Relative Reconstruction Error')
    plot["B"].legend(loc="upper right");
    fig.show()

## Eigenvector Layout

In [ ]:
def evectLayout(sd, vect, threshold=0):
    #x = electrode_mapping.x.values
    #y = electrode_mapping.y.values

    sttc = sd.spike_time_tilings()
    firing_rates = calculate_mean_firing_rates(sd)
    
    neuron_x = []
    neuron_y = []
    #neuron_amp = []
    for neuron in sd.neuron_data[0].values(): # Plots neurons on a 2-d space, representing their positions on the array
    #     print("x,y:",neuron['position'])
        neuron_x.append(neuron['position'][0])

        neuron_y.append(neuron['position'][1])
        #neuron_amp.append(np.mean(neuron['amplitudes']))

    # 
    plt.figure(figsize=(7,5)) 
    ax = plt.axes()
    ax.set_facecolor("grey")
    #plt.scatter(neuron_x,neuron_y, s=neuron_amp, c=vect, cmap = plot_color) # color each plotted neuron according to the values of the eigenvector
    plt.scatter(neuron_x,neuron_y, s=firing_rates*20, c=vect, cmap = plot_color) # color each plotted neuron according to the values of the eigenvector
   
    
    for i in range(sttc.shape[0]): # plot connectivity lines between neurons
        for j in range(sttc.shape[1]):

            # Only need to do upper triangle since sttc' = sttc
            if i<=j: continue

            if sttc[i,j] < threshold : continue

            #Position of neuron i
            ix,iy = sd.neuron_data[0][i]['position']
            jx,jy = sd.neuron_data[0][j]['position']

            # Plot line between the points, linewidth is the sttc
            plt.plot([ix,jx],[iy,jy], linewidth=sttc[i,j],c='k')
            
    plt.xlabel('um')
    plt.ylabel('um')
    plt.title("Neuron layout")
    plt.colorbar()
    plt.show()

### <font color="blue"> Eigenvector Layout Settings

In [ ]:
method_options = ["Correlation", "Spike Time Tiling"]

In [ ]:
def EigenvectorLayout(Method):
    global ELayout_usemethod;
    
    if Method == "Correlation":
        Corr = correlation(sd)
        Wcorr, Ucorr = eigenvalues_eigenvectors(Corr)
        ELayout_usemethod = Ucorr
    else:
        STTC = get_sttc(sd)
        Wsttc, Usttc = eigenvalues_eigenvectors(STTC)
        ELayout_usemethod = Usttc
        
    interact_manual( vectorSelect, Vector=(0,len(ELayout_usemethod),1), thresholdIn=(0,1,0.01)) 

In [ ]:
def vectorSelect(Vector, thresholdIn):
    evectLayout(sd, ELayout_usemethod[:,Vector], thresholdIn)

In [ ]:
def vectorSelectV2(Vector):
    evectLayout(sd, ELayout_usemethod[:,Vector])

# <font color="red">Random Permutations

### Randomizers

In [ ]:
def random_rotation(sd, seedIn=np.random.randint(0, 1000000000)): #seed=np.random.randint(0, 10000)
    # Randomizes a dataset while perseving an underlying structure by stitching sections of the train together at random points
    RotatedTrain = []
    TrainDat = sd.train
    
    np.random.seed(seedIn) # set random seed to seedIn
    seeds = [np.random.randint(0, 1000000000) for _ in range(len(TrainDat))] # creat array of randomized seeds from seedIn
    i = 0
    
    for neuron in TrainDat: # Loop through indiv neurons in train
        neuronTrain = []
        rand.seed(seeds[i]) # set random seed to one of seedIn's generated vals
        i += 1
        alpha = rand.randrange(1, len(neuron)) # Select random index
        
        index = alpha
        prevtime = 0 # global time
        while index < len(neuron): # Loop through neurons after alpha
            firingInst = neuron[index] - neuron[index - 1] # calculate firing time difference
            firingInst += prevtime # add difference to global time
            neuronTrain.append(firingInst)
            prevtime = firingInst # update global time
            index += 1
        index = 0
        splitEndVal = neuronTrain[-1] # get last time value of the split
        while index < alpha: # Loop through before alpha
            neuronTrain.append(neuron[index] + splitEndVal) # add times of pre-alpha neurons to the last time value and append 
            index += 1
        
        RotatedTrain.append(np.array(neuronTrain))
    return SpikeData(RotatedTrain, length=sd.length, N=sd.N, 
                         metadata=sd.metadata, neuron_data=sd.neuron_data,
                         neuron_attributes=sd.neuron_attributes) # convert to spikedata object and return

In [ ]:
def random_shuffle(sd, seedIn=np.random.randint(0, 1000000000)): #seed=np.random.randint(0, 10000)
    # Randomizes a dataset while perseving an underlying structure by swaping firing times between neurons
    Rastered = sd.raster(bin_size=1) # get spikedata's raster
    RasterT = np.transpose(Rastered) # transpose raster to look at individual time slices
    shuffleT = []
    dt=1.0
  
    np.random.seed(seedIn) # set random seed to seedIn
    seeds = [np.random.randint(0, 1000000000) for _ in range(len(RasterT))] # creat array of randomized seeds from seedIn
    i = 0
    
    for timeslice in RasterT:
        randslice = np.copy(timeslice) 
        np.random.seed(seeds[i]) # set local seed to a seedIn generated value for randomized shuffle
        i += 1
        np.random.shuffle(randslice) # shuffle timeslice
        shuffleT.append(randslice) 
        
    ShuffleRaster = np.transpose(shuffleT) # re-transpose raster to get origional orientation 
    idces, times = np.nonzero(ShuffleRaster)
    
    return SpikeData(idces, times*dt, length=sd.length, N=sd.N, 
                         metadata=sd.metadata, neuron_data=sd.neuron_data,
                         neuron_attributes=sd.neuron_attributes)

In [ ]:
def KennethHarris(sd, dt=1.0, seed=None):
        '''
        Create a new SpikeData object which preserves the population
        rate and mean firing rate of each neuron in an existing
        SpikeData by randomly reallocating all spike times to different
        neurons at a resolution given by dt.
        '''
        # Collect the spikes of the original Spikedata and define a new
        # "randomized spike matrix" to store them in.
        sm = sd.sparse_raster(dt)

        idces, times = np.nonzero(randomize_raster(sm, seed))
        return SpikeData(idces, times*dt, length=sd.length, N=sd.N,
                         metadata=sd.metadata, neuron_data=sd.neuron_data,
                         neuron_attributes=sd.neuron_attributes)

### <font color="green">Plot Random Matricies

In [ ]:
def randomPlots(corr, sttc, toep, rcorr, rsttc, rtoep, randMethod):
    figLayout = """ 
                ABC
                DEF
                """
    
    fig, plot = plt.subplot_mosaic(figLayout, figsize=(18, 9))
    fig.tight_layout(pad=5.0)
    
    # Original Plots ------------------------------------------------------------------------------
    # Plot Correlation 
    pltA=plot["A"].imshow(corr[:,:2*len(corr)].T, interpolation='none', cmap=plot_color) 
    plot["A"].set_title("Correlation Eigenvectors")
    plot["A"].set_ylabel('Eigenvector Number')
    plot["A"].set_xlabel('Observation Dimension')
    fig.colorbar(pltA, ax=plot["A"])
    
    # Plot Spike Time Tiling
    pltB=plot["B"].imshow(sttc[:,:2*len(sttc)].T, interpolation='none', cmap=plot_color) 
    plot["B"].set_title("STTC Eigenvectors")
    plot["B"].set_ylabel('Eigenvector Number')
    plot["B"].set_xlabel('Observation Dimension')
    fig.colorbar(pltA, ax=plot["B"])
    
    # Plot Toeplitz 
    pltC=plot["C"].imshow(toep[:,:2*len(toep)].T, interpolation='none', cmap=plot_color) 
    plot["C"].set_title("Toeplitz Eigenvectors")
    plot["C"].set_ylabel('Eigenvector Number')
    plot["C"].set_xlabel('Observation Dimension')
    fig.colorbar(pltA, ax=plot["C"])
    
    # Random Plots --------------------------------------------------------------------------------
    # Plot Random Correlation 
    pltD=plot["D"].imshow(rcorr[:,:2*len(rcorr)].T, interpolation='none', cmap=plot_color) 
    plot["D"].set_title("Random Correlation Eigenvectors")
    plot["D"].set_ylabel('Eigenvector Number')
    plot["D"].set_xlabel('Observation Dimension')
    fig.colorbar(pltA, ax=plot["D"])
    
    # Plot Random STTC
    pltE=plot["E"].imshow(rsttc[:,:2*len(rsttc)].T, interpolation='none', cmap=plot_color) 
    plot["E"].set_title("Random STTC Eigenvectors")
    plot["E"].set_ylabel('Eigenvector Number')
    plot["E"].set_xlabel('Observation Dimension')
    fig.colorbar(pltA, ax=plot["E"])
    
    # Plot Toeplitz
    pltF=plot["F"].imshow(rtoep[:,:2*len(rtoep)].T, interpolation='none', cmap=plot_color) 
    plot["F"].set_title("Random Toeplitz Eigenvectors")
    plot["F"].set_ylabel('Eigenvector Number')
    plot["F"].set_xlabel('Observation Dimension')
    fig.colorbar(pltA, ax=plot["F"])
    
    fileNoExt = file.split(".zip")[0]
    snapshot = datetime.now(Timezone)
    current_time = snapshot.strftime("%Y-%m-%d_%H-%M-%S")
    savepath = "/home/jovyan/work/Human_Hippocampus/Ari/Randomization/" + f"{randMethod}_{fileNoExt}_from_{folder}_taken_{current_time}sec.png"
    #plt.savefig(savepath)

In [ ]:
def plotRandEvectLayout(Method): 
    global Rand_ELayout_usemethod;
    
    if Method == "Correlation":
        Corr = correlation(rsd)
        Wcorr, Ucorr = eigenvalues_eigenvectors(Corr)
        Rand_ELayout_usemethod = Ucorr
    else:
        STTC = get_sttc(rsd)
        Wsttc, Usttc = eigenvalues_eigenvectors(STTC)
        Rand_ELayout_usemethod = Usttc
        
    interact_manual( RandVectorSelect, Vector=(0,5,1),  thresholdIn=(0,1,0.01)) 

In [ ]:
def RandVectorSelect(Vector, thresholdIn):
    evectLayout(rsd, Rand_ELayout_usemethod[:,Vector], thresholdIn)

In [ ]:
def PlotRandomBasis(): 
    Corr = correlation(sd)
    STTC = get_sttc(sd)
    
    randCorr = correlation(rsd)
    randSTTC = get_sttc(rsd)
    
    Wcorr, Ucorr = eigenvalues_eigenvectors(Corr)
    Wsttc, Usttc = eigenvalues_eigenvectors(STTC)
    
    WRcorr, URcorr = eigenvalues_eigenvectors(randCorr)
    WRsttc, URsttc = eigenvalues_eigenvectors(randSTTC)
    
    methodList = [Ucorr, URcorr, Usttc, URsttc]
    
    fig, plot = plt.subplots(nrows=5, ncols=4, figsize=(20,9))
    fig.suptitle("First Five Eigenvectors of Data and Randomized Data")
    
    for i in range(5):
        for j in range(4):
            plot[i, j].stem(methodList[j][:,i])
            
            if i==4:
                plot[i,j].set_xlabel('Observation Dimension')
            if i==0:
                if j == 0:
                    plot[i,j].set_title('Dataset Correlation ')
                elif j == 1:
                    plot[i,j].set_title('Random Correlation')
                elif j == 2:
                    plot[i,j].set_title('Dataset STTC')
                else:
                    plot[i,j].set_title('Random STTC')

In [ ]:
def PlotRandomCheck():
    Corr = correlation(sd)
    STTC = get_sttc(sd)
    
    randCorr = correlation(rsd)
    randSTTC = get_sttc(rsd)
    
    figLayout = """
                AB
                CD
                """
    
    fig, plot = plt.subplot_mosaic(figLayout, figsize=(14,7)) 
    fig.tight_layout(pad=5.0)
    
    # Plot Correlation Matrix
    pltA = plot["A"].imshow(Corr, cmap=plot_color) # plot STTC matrix
    plot["A"].set_title("Correlation")
    plot["A"].set_xlabel("Neuron index")
    plot["A"].set_ylabel("Neuron index" )
    fig.colorbar(pltA, ax=plot["A"], shrink=0.7)

    # Plot STTC matrix
    pltB = plot["B"].imshow(STTC, cmap=plot_color) # plot STTC matrix
    plot["B"].set_title("STTC")
    plot["B"].set_xlabel("Neuron index")
    plot["B"].set_ylabel("Neuron index" )
    fig.colorbar(pltB, ax=plot["B"], shrink=0.7)
    
    # Plot random Correlation matrix
    pltC = plot["C"].imshow(randCorr, cmap=plot_color) # plot STTC matrix
    plot["C"].set_title("Random Correlation")
    plot["C"].set_xlabel("Neuron index")
    plot["C"].set_ylabel("Neuron index" )
    fig.colorbar(pltC, ax=plot["C"], shrink=0.7)

    # Plot random STTC matrix
    pltD = plot["D"].imshow(randSTTC, cmap=plot_color) # plot STTC matrix
    plot["D"].set_title("Random STTC")
    plot["D"].set_xlabel("Neuron index")
    plot["D"].set_ylabel("Neuron index" )
    fig.colorbar(pltD, ax=plot["D"], shrink=0.7)

### <font color="blue">Random Settings

In [ ]:
randMethods = ["Rotation", "Shuffle", "Kenneth Harris"]

In [ ]:
def RandomiseDataset(method, seed, resolution):
    global rsd
    
    # Options Select
    if method=="Shuffle":
        rsd = random_shuffle(sd, seed)
        #rsd = sd.randomized() 
        titleMethod = "shuffled"
        
    elif method=="Kenneth Harris":
        rsd = KennethHarris(sd, dt=resolution, seed=seed)
        titleMethod = "KH"
        
    else:
        rsd = random_rotation(sd, seed) 
        titleMethod = "rotated"
    
    Corr = correlation(sd)
    STTC = get_sttc(sd)
    Toep = toeplitz(Corr)
    
    randCorr = correlation(rsd)
    randSTTC = get_sttc(rsd)
    randToep = toeplitz(randCorr)
    
    Wcorr, Ucorr = eigenvalues_eigenvectors(Corr)
    Wsttc, Usttc = eigenvalues_eigenvectors(STTC)
    Wt, Ut = eigenvalues_eigenvectors(Toep)
    
    WRcorr, URcorr = eigenvalues_eigenvectors(randCorr)
    WRsttc, URsttc = eigenvalues_eigenvectors(randSTTC)
    WRt, URt = eigenvalues_eigenvectors(randToep)
    
    randomPlots(Ucorr, Usttc, Ut, URcorr, URsttc, URt, titleMethod)

# <font color="red">Bookend

In [ ]:
now = datetime.now(Timezone)
printNow = now.strftime("%Y/%m/%d %H:%M:%S")

print(f"Done at: {printNow}")